In [55]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

llm_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model = "gemini-1.5-flash")


In [56]:
result = llm_model.invoke("what is rag in AI in 10 words")
print(result)

content='Retrieval-Augmented Generation: AI using external knowledge for improved responses.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-60b8c203-1c10-47b7-bc22-54a8084a6627-0' usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


# Load PDF

In [57]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("pdfs\case study-2.pdf")
documents = loader.load_and_split()
documents

[Document(metadata={'source': 'pdfs\\case study-2.pdf', 'page': 0}, page_content='UNIVERSITY CASEBOOK SERIES® \nLEGAL ETHICS CASE STUDIES EIGHTH EDITION     DEBORAH L. RHODE Ernest W. McFarland Professor of Law Director, Stanford Center on the Legal Profession Stanford Law School DAVID LUBAN University Professor Professor of Law and Philosophy Georgetown University Law Center SCOTT L. CUMMINGS Robert Henigson Professor of Legal Ethics Professor of Law UCLA Law School NORA FREEMAN ENGSTROM Professor of Law Deane F. Johnson Faculty Scholar Stanford Law School      \n  © 2020'),
 Document(metadata={'source': 'pdfs\\case study-2.pdf', 'page': 1}, page_content='TABLE OF CONTENTS     Preface……………………………………………………………1  Case Study 1: Rita’s Case……………………………………...5  Case Study 2: David Boies’s Work for Harvey Weinstein in #MeToo………………………………………………….31  Case Study 3: ACLU Counsel Representing an  Undocumented Minor Seeking an Abortion………….41  Case Study 4: Guantánamo Defense Counsel……………….67'),
 Doc

# pages of pdf

In [58]:
# Extract the text content from the Document objects
pages = [doc.page_content for doc in documents]
pages


['UNIVERSITY CASEBOOK SERIES® \nLEGAL ETHICS CASE STUDIES EIGHTH EDITION     DEBORAH L. RHODE Ernest W. McFarland Professor of Law Director, Stanford Center on the Legal Profession Stanford Law School DAVID LUBAN University Professor Professor of Law and Philosophy Georgetown University Law Center SCOTT L. CUMMINGS Robert Henigson Professor of Legal Ethics Professor of Law UCLA Law School NORA FREEMAN ENGSTROM Professor of Law Deane F. Johnson Faculty Scholar Stanford Law School      \n  © 2020',
 'TABLE OF CONTENTS     Preface……………………………………………………………1  Case Study 1: Rita’s Case……………………………………...5  Case Study 2: David Boies’s Work for Harvey Weinstein in #MeToo………………………………………………….31  Case Study 3: ACLU Counsel Representing an  Undocumented Minor Seeking an Abortion………….41  Case Study 4: Guantánamo Defense Counsel……………….67',
 'Preface \nIn real-life law practice, ethical issues do not always come neatly \nboxed and labeled. Often, a representation will raise multiple ethical \nissues, and

# text splitter

In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter , CharacterTextSplitter

text_split = CharacterTextSplitter(chunk_size=100, chunk_overlap=50)
chunks = text_split.create_documents(pages)

# Display results
print(f"Number of chunks: {len(chunks)}")
print(f"Type of first chunk: {type(chunks[0])}")

Number of chunks: 84
Type of first chunk: <class 'langchain_core.documents.base.Document'>


# Embeddings

In [60]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key= GOOGLE_API_KEY,  model="models/text-embedding-004",  chunk_size=100,task_type="retrieval_document")


embedding_model


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000101C5E25100>, model='models/text-embedding-004', task_type='retrieval_document', google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

# faiss vector-db

In [62]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_texts(pages, embedding=embedding_model)
vector_store.save_local("faiss_index")

In [63]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


# Prompt template

In [64]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate , HumanMessagePromptTemplate


In [65]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are an expert Legal Research Assistant specializing in case law analysis and legal drafting.
                 Given case law context and a question from the user, you should:
                 1. Analyze the legal principles and precedents from the provided cases
                 2. Identify relevant holdings and ratio decidend
                 3. Provide well-reasoned legal analysis with proper citations
                 4. When drafting, follow proper legal writing conventions and structure
                 
                 Format your responses with:
                 - Clear headings for different sections
                 - Proper case citations
                 - Specific references to relevant passages from the context
                 - Clear distinction between holdings and obiter dicta
                 - Practical applications or implications when relevant"""),
    
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Analyze the following legal materials and answer the question provided.
    
    Case Law Context: {context}
    
    Legal Query: {question}
    
    Analysis: """)
])

In [68]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [70]:
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime
def format_legal_docs(docs):
    """Format legal documents with proper structure and citations"""
    formatted_docs = []
    for i, doc in enumerate(docs, 1):
        # Extract metadata if available
        citation = doc.metadata.get('citation', 'No citation available')
        date = doc.metadata.get('date', 'No date available')
        court = doc.metadata.get('court', 'Court information not available')
        
        # Format each document with legal context
        formatted_text = f"""
Case Document {i}
Citation: {citation}
Court: {court}
Date: {date}
---
{doc.page_content}
---"""
        formatted_docs.append(formatted_text)
    
    return "\n\n".join(formatted_docs)

# Define the RAG chain with legal processing
rag_chain = (
    {
        "context": retriever | format_legal_docs,
        "question": RunnablePassthrough(),
        "metadata": lambda x: {
            "query_type": "legal_analysis",
            "timestamp": datetime.now().isoformat()
        }
    }
    | chat_template
    | llm_model
    | output_parser
)

In [75]:
response = rag_chain.invoke("summarize the Rita’s Case ")
print(response)

## Summary of Rita's Case

Rita's Case, as presented in the provided excerpts, details the legal and social service involvement surrounding Rita, a young girl with a troubled family history.  Born to a heroin-addicted mother, Rita spent her early years in the care of her maternal grandmother, Gladys. Gladys, facing challenges in managing Rita's increasingly aggressive and difficult behavior, repeatedly sought assistance from the Department of Public Welfare (DPW).

The DPW's interventions involved placing Rita in the Catholic Home for Children, initially as a therapeutic placement. However, the Home, often used as a clearinghouse for foster care, eventually attempted to place Rita with a foster family.  This placement was unsuccessful due to strained relations between Gladys and the foster parents, leading to Rita's return to her grandmother.

Subsequent difficulties led to another DPW intervention and Rita's return to the Catholic Home.  Concerns arose about Rita's apparent sexual sop